In [2]:
import pandas as pd
import numpy as np
from numpy.core.records import fromarrays
from scipy.io import savemat

/tmp/ipykernel_2486/495685066.py:3: DeprecationWarning: numpy.core.records is deprecated and has been renamed to numpy._core.records. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.records.fromarrays.
  from numpy.core.records import fromarrays


In [3]:
choice = 0  #start from 0
devices = ['nfet_03v3', 'pfet_03v3']

# widths used for characterization
w = np.array([5.0, 5.0, 5.0, 5.0])
nfing = np.array([1, 1, 1, 1])

In [4]:
# read ngspice data
df_raw = pd.read_csv('/foss/designs/SSCS-Chipathon-2025-DAC-for-the-future/DESIGN/DACFUTURE_EDU/sweep/sweep_script/techsweep_'+devices[choice]+'.txt', sep='\s+')
par_names = df_raw.columns.to_list()
fet_name = par_names[1].split('[')[0]

# remove unwanted columns and rename for readability
df = df_raw.drop(['frequency', 'frequency.1'], axis=1)
df = df.apply(pd.to_numeric)
df.columns = df.columns.str.replace(fet_name, '')
df.columns = df.columns.str.replace(fet_name[1:], '')
df.columns = df.columns.str.replace('[dc]', '')
df.columns = df.columns.str.replace('onoise..', 'n')
df.columns = df.columns.str.removeprefix('@')
df.columns = df.columns.str.removeprefix('[')
df.columns = df.columns.str.removesuffix(']')
df

<>:2: SyntaxWarning: invalid escape sequence '\s'
<>:2: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_2486/339892717.py:2: SyntaxWarning: invalid escape sequence '\s'
  df_raw = pd.read_csv('/foss/designs/SSCS-Chipathon-2025-DAC-for-the-future/DESIGN/DACFUTURE_EDU/sweep/sweep_script/techsweep_'+devices[choice]+'.txt', sep='\s+')
/tmp/ipykernel_2486/339892717.py:2: SyntaxWarning: invalid escape sequence '\s'
  df_raw = pd.read_csv('/foss/designs/SSCS-Chipathon-2025-DAC-for-the-future/DESIGN/DACFUTURE_EDU/sweep/sweep_script/techsweep_'+devices[choice]+'.txt', sep='\s+')


FileNotFoundError: [Errno 2] No such file or directory: '/foss/designs/SSCS-Chipathon-2025-DAC-for-the-future/DESIGN/DACFUTURE_EDU/sweep/sweep_script/techsweep_nfet_03v3.txt'

In [ ]:
# sweep variable vectors
l =   np.round(np.unique(df['l'])*1e6, 2)
vgs = np.unique(df['vg'])
vds = np.unique(df['vd'])
vsb = np.unique(-df['vb'])

In [ ]:
# ngspice sweep order is l, vgs, vds, vsb
dims = [len(l), len(vgs), len(vds), len(vsb)]
id = np.reshape(df['id'].values, dims)
vt = np.reshape(df['vth'].values, dims)
gm = np.reshape(df['gm'].values, dims)
gmb = np.reshape(df['gmbs'].values, dims)
gds = np.reshape(df['gds'].values, dims)
cgg = np.reshape(df['cgg'].values, dims) \
      + np.reshape(df['cgdo'].values, dims) + np.reshape(df['cgso'].values, dims)
cgb = -np.reshape(df['cgb'].values, dims)
cgd = -np.reshape(df['cgd'].values, dims) \
      + np.reshape(df['cgdo'].values, dims)
cgs = -np.reshape(df['cgs'].values, dims) \
      + np.reshape(df['cgso'].values, dims)
cdd = np.reshape(df['cdd'].values, dims) \
      + np.reshape(df['capbd'].values, dims) \
      + np.reshape(df['cgdo'].values, dims)
css = np.reshape(df['css'].values, dims) \
      + np.reshape(df['capbs'].values, dims) \
      + np.reshape(df['cgso'].values, dims)
sth = np.reshape(df['nid'].values, dims)**2
sfl = np.reshape(df['n1overf'].values, dims)**2


In [ ]:
dic = {
  "INFO": "GlobalFoundries, 180nm MCU CMOS , BSIM4",
  "CORNER": "NOM",
  "TEMP": 300.0,
  "VGS": vgs,
  "VDS": vds,
  "VSB": vsb,
  "L": l,
  "W": w[choice],
  "NFING": nfing[choice],
  "ID": id,
  "VT": vt,
  "GM": gm,
  "GMB": gmb,
  "GDS": gds,
  "CGG": cgg,
  "CGB": cgb,
  "CGD": cgd,
  "CGS": cgs,
  "CDD": cdd,
  "CSS": css,
  "STH": sth,
  "SFL": sfl
}
savemat('/foss/designs/SSCS-Chipathon-2025-DAC-for-the-future/DESIGN/DACFUTURE_EDU/sweep/sweep_script/'+devices[choice]+'.mat', {devices[choice]: dic})